# Machine Learning COS-623 - Terceiro Trimestre de 2017
## Terceira e Quarta Listas de Exercı́cios (Graduação e Pós-Graduação)



In [ ]:
import numpy as np # manipulação de matrizes
import pandas as pd # Manipulação de datasets
import matplotlib.pyplot as plt # Para plotagem
import seaborn as sns # Coloração melhor nas plotagens
import scipy.stats as stats # Normalizações, cálculos de densidade de probabilidade, etc
import sklearn # Implementação de modelos preditivos e de classificação

%matplotlib inline

In [ ]:
## Lendo o dataset e separando os dados em 1000 dados de treino e 172 dados de teste
df = pd.read_csv("Dados-medicos.txt",sep=" ")
df = df.drop('Unnamed: 1', 1) ## removendo coluna gerado errôneamentod devido à formatação dos dados
df = df.drop('Unnamed: 2', 1) ## removendo coluna gerado errôneamentod devido à formatação dos dados

train = df.sample(frac = 0.85324232082,random_state=200)
test  = df.drop(train.index)

### Funções de apoio para primeira questão

Vamos definir uma função para executar o modelo de Regressão Linear, bem como analisar seu desempenho ao manipular as variáveis de entrada X e também plotar seu resultado final.

In [ ]:
from sklearn.linear_model import LinearRegression ## Modelo de regressão linear
from sklearn.preprocessing import PolynomialFeatures ## Geração de features para modelos

def do_rmse(y,y_predicted):
    return np.sqrt(np.mean((y - y_predicted) **2 ))

def negativeLogLikelihood(testDataset,variance,y,y_predicted):
    return -(len(testDataset)/2)*np.log(2*np.pi*variance) - (1/(2*variance))* np.sum((y - y_predicted) **2 )

def linearRegression2(train,test,variables,target,polyDegree, toPlot = False, plotedVariables = None):
    # Vamos criar trainFeatures e testFeatures e aplicar sobre eles as transformações dados pelas base functions
    # E.g., Para duas features a e b, para um polinomio de grau 2, as funções base serão:
    # [1, a, b, a^2, ab, b^2]

    trainData = train[variables]
    trainFeatures = trainData.drop(target, axis = 1)

    testData = test[variables]
    testFeatures = testData.drop(target, axis = 1)

    polyDegree = 4

    # executa para todos os graus de 1 até polyDegree
    for i in range (1,polyDegree+1):
        poly = PolynomialFeatures(degree=i)
        X = poly.fit_transform(trainFeatures)
        test_X = poly.fit_transform(testFeatures)

        X = pd.DataFrame(data=X[0:,0:],index=X[0:,0])
        test_X = pd.DataFrame(data=test_X[0:,0:],index=test_X[0:,0]) 

        lm = LinearRegression()
        lm.fit(X,trainData.VO2_medido_máximo)
        print(pd.DataFrame(list(zip(X.columns,lm.coef_)),columns = ['features','w']))
        rmse =  np.sqrt(np.mean( (test.VO2_medido_máximo - lm.predict(test_X) ) **2 ))
        print("O RMSE com polinômio de grau %i foi de %f" % (i,rmse)) 
        var = test[['VO2_medido_máximo']].var()
        nll = -(len(test)/2)*np.log(2*np.pi*var) - (1/(2*var))* np.sum( (test.VO2_medido_máximo - lm.predict(test_X) ) **2 )
        print("O NLL com polinômio de grau %i foi de %f \n" % (i,nll)) 
        
    if(toPlot):
        # plots graph
        y = train[['VO2_medido_máximo']].values
        x = train[plotedVariables].values
        plt.scatter(x,y)
        plt.plot(x, lm.predict(poly.fit_transform(x)), color='blue', linewidth=3)

        plt.grid(True)
        plt.savefig("test.png")
        plt.show()

## Questão 1

### 1.1

In [ ]:
linearRegression2(train,test,['Carga_Final','VO2_medido_máximo'],['VO2_medido_máximo'],4, True)

### Questão 1.2

In [ ]:
linearRegression2(
    train,
    test,
    ['Peso', 'Carga_Final','VO2_medido_máximo'],
    ['VO2_medido_máximo'],
    4
)

### Questão 1.3

In [ ]:
linearRegression2(
    train,
    test,
    ['IDADE', 'Peso', 'Carga_Final','VO2_medido_máximo'],
    ['VO2_medido_máximo'],
    4
)

## Questao 2

### 2.1 

Referências:

- https://en.wikipedia.org/wiki/Multivariate_normal_distribution
- https://stackoverflow.com/questions/38698277/plot-normal-distribution-in-3d

In [ ]:
from scipy.stats import multivariate_normal
from mpl_toolkits.mplot3d import Axes3D

# Parâmetros da Gaussiana

var_carga = train[['Carga_Final']].var().values[0]
mean_carga = train[['Carga_Final']].mean().values[0]

var_vo2 = train[['VO2_medido_máximo']].var().values[0]
mean_vo2 = train[['VO2_medido_máximo']].mean().values[0]

## Criação de Grid para plotagem da Gaussiana multivariada 

x = np.linspace(train[['Carga_Final']].min(),train[['Carga_Final']].max(),500)
y = np.linspace(train[['VO2_medido_máximo']].min(),train[['VO2_medido_máximo']].max(),500)

X, Y = np.meshgrid(x,y)
pos = np.empty(X.shape + (2,))
pos[:, :, 0] = X; pos[:, :, 1] = Y
rv = multivariate_normal([mean_carga, mean_vo2], [[var_carga, 0], [0, var_vo2]])

# Plotagem 3D

fig = plt.figure(figsize=(9, 9))
ax = fig.gca(projection='3d')
ax.plot_surface(X, Y, rv.pdf(pos),cmap='viridis',linewidth=0)
ax.set_xlabel('Carga_Final')
ax.set_ylabel('VO2_medido_máximo')
ax.set_zlabel('Densidade de Probabilidade')
plt.show()

### 2.2

Dado que a variável *V02* é contínua, a maneira mais adequada para resolver esse problema é primeiro integrando nossa Gaussiana no eixo do **Peso** e da **Carga** de acordo com os valores dados, depois definindo faixas de intervalo em V02 e integrando  para encontrar qual obtém maior probabilidade.

Devido a limitações de capacidade de processamento, neste trabalho nós retornaremos apenas uma aproximação do valor mais provável de VO2, sem a distribuição de probabilidade dos demais valores.

In [ ]:
def acsm_formula(w,peso):
    return round((w*11.4+260+peso*3.5)/peso,3)

In [ ]:
mean_peso = train[['Peso']].mean().values[0]

my_data = train[['Carga_Final','VO2_medido_máximo','Peso']]
cov = my_data.cov().as_matrix()

rv2 = multivariate_normal([mean_carga,mean_peso,mean_vo2], cov)

sample_values =  [[70,94.1],[260,70],[220,100]]

for j in range(len(sample_values)):
    probs = []
    max_prob = 0 
    max_value = 0
    values = np.linspace(train[['VO2_medido_máximo']].min(),train[['VO2_medido_máximo']].max(),10000)
    for i in values:
        vo2 = rv2.pdf([sample_values[j][0],sample_values[j][1],i])
        probs.append(vo2)
        if(vo2 > max_prob):
            max_prob = vo2
            max_value = i 
    print(("O valor de Vo2 mais provável para o peso %s e a carga %s é: %s e o valor previsto pela ACMS é de %s") %
          (sample_values[j][0],
           sample_values[j][1],
           round(max_value,3),
           acsm_formula(sample_values[j][0],sample_values[j][1])))

### 2.3

In [ ]:
mean_idade = train[['IDADE']].mean().values[0]

my_data = train[['Carga_Final','VO2_medido_máximo','Peso','IDADE']]
cov = my_data.cov().as_matrix()

rv2 = multivariate_normal([mean_carga,mean_peso,mean_idade,mean_vo2], cov)

sample_values =  [[70,94.1,18],[260,70,30],[220,100,60]]

for j in range(len(sample_values)):
    probs = []
    max_prob = 0 
    max_value = 0
    values = np.linspace(train[['VO2_medido_máximo']].min(),train[['VO2_medido_máximo']].max(),10000)
    for i in values:
        vo2 = rv2.pdf([sample_values[j][0],sample_values[j][1],sample_values[j][2],i])
        probs.append(vo2)
        if(vo2 > max_prob):
            max_prob = vo2
            max_value = i 
    print(("O valor de Vo2 mais provável para o peso %s,a carga %s e idade %s é: %s e o valor previsto pela ACMS é de %s") %
          (sample_values[j][0],
           sample_values[j][1],
           sample_values[j][2],
           round(max_value,3),
           acsm_formula(sample_values[j][0],sample_values[j][1])))

## Questão 3
### Questão 3.1
Haverá diferença somente se a prior tiver pesos diferentes? (VERIFICAR E DISSERTAR MELHOR)

In [ ]:
# Parâmetros da Gaussiana
train = train[['Carga_Final']].var().values[0]
mean_carga = train[['Carga_Final']].mean().values[0]

### Questão 3.2
Onde $\boldsymbol{x} = [x_1, x_2, .., x_D]$
, pela independência entre as features assumida no modelo Naive Bayes, podemos calcular a probabilidade da classe como:
$$ p(h = c \mid \boldsymbol{x}, \theta) \propto p(h=c \mid \theta) \prod_{i=1}^{D} p(x_i \mid h=c, \theta)  $$
$$ posterior \propto prior * likelihood $$
A prior $p(h=c \mid \theta)$ que pode ser genarada como o número de amostras pertencentes a classe $c$ dividido pelo número total de amostras.

A likelihood, portanto, é baseado no modelo Gaussiano, já presente no enunciado da questão.

### Questão 3.3
A suposição básica do Naive Bayes é assumir que as features sejam independêntes entre si, o que não é
necessário no modelo de Gaussiana Multivariada.

### Questão 3.4
Temos que a likelihood é: 
$$\mathcal{L}(\theta \mid \boldsymbol{x}, y=c) = P(\boldsymbol{x}, y=c \mid \theta) \propto P(\boldsymbol{x} \mid y=c) P(y=c \mid \theta)$$
Pela independência das features:
$$\mathcal{L}(\theta \mid \boldsymbol{x}, y=c) = P(y=c \mid \theta) \prod_{i=1}^{D} P(x_i \mid y=c, \theta)$$
Aplicando log:
$$log \mathcal{L}(\theta \mid \boldsymbol{x}, y=c) = log(P(y=c \mid \theta)) + \sum_{i=1}^{D} log(P(x_i \mid y=c, \theta))$$
Para otimizar os parâmetros, precisamos derivar em relação ao parâmetro e igualar a zero.

## Questão 4

In [ ]:
from sklearn import mixture

df = train[['Carga_Final','VO2_medido_máximo','Peso']]
samples = test[['Carga_Final','VO2_medido_máximo','Peso']]

gmm = mixture.GaussianMixture(n_components=3)
gmm.fit(np.array(df))
labels = gmm.predict(np.array(samples))
plt.scatter(np.array(samples)[:, 0], np.array(samples)[:, 2], c=labels, s=40, cmap='viridis');
